In [2]:
!pip install pymysql

In [1]:
import pymysql

In [52]:
class DataBase:
    def __init__(self):
        self.connection = pymysql.connect(
            host = "localhost",
            user = "root",
            password = "",
            db = "teleton"
        )
        self.cursor = self.connection.cursor()
    
        print("Conexión exitosa")
    
    def insert(self, tabla, nomina, nombre, puesto, tags, certificado, estatus):
            sql = f"INSERT INTO {tabla} (`Nomina`, `Nombre`, `Puesto`, `Tags`, `Certificado`, `Estatus`) VALUES ('{nomina}', '{nombre}', '{puesto}', '{tags}', '{certificado}', '{estatus}')"
            print(sql)
            try:
                self.cursor.execute(sql)
                print(f"{nombre} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
    def select(self, tabla, what, _from, where, value):
        if what == "*":
            sql = f"SELECT {what} FROM {_from}"
        else:
            sql = f"SELECT {what} FROM {_from} where {where} = '{value}'"
        print(sql)
        self.cursor.execute(sql)
        val = self.cursor.fetchall()
        return val
        
    def update(self, tabla, what, value, where_col, where_val):
        sql = f"UPDATE {tabla} set {what} = '{value}' where {where_col} = '{where_val}'"
        self.cursor.execute(sql)
        print("Actualización exitosa")
        
    def delete(self, tabla, where, value):
        sql = f"DELETE FROM {tabla} where {where} = '{value}'"
        self.cursor.execute(sql)
        print("Eliminación exitosa")

database = DataBase()

Conexión exitosa


In [29]:
database.insert("users","A0175068", "Jairo", "Estudiante", "IDM", "pdjsnem", "Activo")

INSERT INTO users (`Nomina`, `Nombre`, `Puesto`, `Tags`, `Certificado`, `Estatus`) VALUES ('A0175068', 'Jairo', 'Estudiante', 'IDM', 'pdjsnem', 'Activo')
Jairo ha sido añadido a la base de datos.


In [56]:
database.select(tabla="users", what = "*", _from = "users", where = "Nomina", value = "A0175043")

SELECT * FROM users


(('A0175043', 'enrique', 'Estudiante', 'IDM', 'gshdjhnbsjcjdnlksd', 'Activo'),
 ('A0175068', 'Jairo', 'Estudiante', 'IDM', 'pdjsnem', 'Activo'))

In [55]:
database.update(tabla = "users", what = "Nombre", value="Juanito", where_col = "Nomina", where_val = "A01752068")

Actualización exitosa


In [53]:
database.delete(tabla = "users", where = "Nomina", value="A01752068")

Eliminación exitosa


In [80]:
from OpenSSL import crypto, SSL

def cert_gen(
    emailAddress="emailAddress",
    commonName="commonName",
    countryName="NT",
    localityName="localityName",
    stateOrProvinceName="stateOrProvinceName",
    organizationName="organizationName",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds=0,
    validityEndInSeconds=10*365*24*60*60,
    KEY_FILE = "private.key",
    CERT_FILE="selfsigned.crt"):
    #can look at generated file using openssl:
    #openssl x509 -inform pem -in selfsigned.crt -noout -text
    # create a key pair
    k = crypto.PKey()
    k.generate_key(crypto.TYPE_RSA, 4096)
    # create a self-signed cert
    cert = crypto.X509()
    cert.get_subject().C = countryName
    cert.get_subject().ST = stateOrProvinceName
    cert.get_subject().L = localityName
    cert.get_subject().O = organizationName
    cert.get_subject().OU = organizationUnitName
    cert.get_subject().CN = commonName
    cert.get_subject().emailAddress = emailAddress
    cert.set_serial_number(serialNumber)
    cert.gmtime_adj_notBefore(0)
    cert.gmtime_adj_notAfter(validityEndInSeconds)
    cert.set_issuer(cert.get_subject())
    cert.set_pubkey(k)
    cert.sign(k, 'sha512')
    with open(CERT_FILE, "wt") as f:
        f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert).decode("utf-8"))
    with open(KEY_FILE, "wt") as f:
        f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, k).decode("utf-8"))

In [93]:
cert_gen(emailAddress="A01750443@tec.mx",
    commonName="tec.mx",
    countryName="MX",
    localityName="Monterrey",
    stateOrProvinceName="Nuevo León",
    organizationName="Tecnológico de Monterrey",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = "privateJERS_2.key",
    CERT_FILE="CertificadoJERS_2.crt")

In [62]:
import OpenSSL.crypto 
cert = OpenSSL.crypto.load_certificate(
    OpenSSL.crypto.FILETYPE_PEM, 
    open('CertificadoJERS.crt').read()
)
cert.get_notBefore()

b'20220407134553Z'

In [88]:
OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, 'CertificadoJERS.crt')

Error: [('PEM routines', 'get_name', 'no start line')]

In [77]:
print(cert.get_signature_algorithm()) #Con qué algoritmo fue firmado
print(cert.get_notAfter()) #fecha de expiración
print(cert.get_pubkey())
print(cert.get_subject()) # Datos sobre el certificado

b'sha512WithRSAEncryption'
b'20230407134553Z'
<X509Name object '/C=MX/ST=Nuevo Le\xC3\xB3n/L=Monterrey/O=Tecnol\xC3\xB3gico de Monterrey/OU=organizationUnitName/CN=tec.mx/emailAddress=A01750443@tec.mx'>


In [91]:
def check_associate_cert_with_private_key(cert, private_key):
    """
    :type cert: str
    :type private_key: str
    :rtype: bool
    """
    try:
        private_key_obj = OpenSSL.crypto.load_privatekey(OpenSSL.crypto.FILETYPE_PEM, open(private_key).read())
    except OpenSSL.crypto.Error:
        raise Exception('private key is not correct: %s' % private_key)

    try:
        cert_obj = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, open(cert).read())
    except OpenSSL.crypto.Error:
        raise Exception('certificate is not correct: %s' % cert)

    context = OpenSSL.SSL.Context(OpenSSL.SSL.TLSv1_METHOD)
    context.use_privatekey(private_key_obj)
    context.use_certificate(cert_obj)
    try:
        context.check_privatekey()
        return True
    except OpenSSL.SSL.Error:
        return False

In [96]:
check_associate_cert_with_private_key("CertificadoJERS_2.crt", "privateJERS.key")

False

In [5]:
import sys
import hashlib

# BUF_SIZE is totally arbitrary, change for your app!
BUF_SIZE = 65536  # lets read stuff in 64kb chunks!

md5 = hashlib.md5()
sha1 = hashlib.sha256()
#hashlib.sha512

with open("../LSMLatinxAI.pdf", 'rb') as f:
    while True:
        data = f.read(BUF_SIZE)
        if not data:
            break
        md5.update(data)
        sha1.update(data)

print("MD5: {0}".format(md5.hexdigest()))
print("SHA1: {0}".format(sha1.hexdigest()))

MD5: d7a4be0864d8b40a1fe18c6139b4807a
SHA1: 7f042c1c98d9beb8ad45d803cfdce6dc25882eced8820ed91b19194b7e19e55e


In [8]:
!pip install pycrypto

  Using cached pycrypto-2.6.1.tar.gz (446 kB)
  Running setup.py clean for pycrypto
Failed to build pycrypto
    Running setup.py install for pycrypto: started
    Running setup.py install for pycrypto: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Jairo Enrique\.conda\envs\DataScience\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Jairo Enrique\\AppData\\Local\\Temp\\pip-install-oyp89_8f\\pycrypto\\setup.py'"'"'; __file__='"'"'C:\\Users\\Jairo Enrique\\AppData\\Local\\Temp\\pip-install-oyp89_8f\\pycrypto\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Jairo Enrique\AppData\Local\Temp\pip-wheel-hh72hveu'
       cwd: C:\Users\Jairo Enrique\AppData\Local\Temp\pip-install-oyp89_8f\pycrypto\
  Complete output (153 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\Crypto
  copying lib\Crypto\pct_warnings.py -> build\lib.win-amd64-3.8\Crypto
  copying lib\Crypto\__init__.py -> bui

In [ ]:
!pip uninstall crypto
!pip uninstall pycrypto
!pip uninstall pycryptodome

In [ ]:
import sys
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA256
from Crypto.Signature import PKCS1_v1_5

def usage():
    print("Usage: \n"
            "digi-sig -s  <priv-key> <data> <signature-file> \n"
            "digi-sig -v  <PUB-key> <data> <signature-file> \n")

if (len(sys.argv) < 5):
    usage()
    quit()

op = sys.argv[1]
key_f = sys.argv[2]
data_f = sys.argv[3]
sig_f = sys.argv[4]

def generate_signature(key, data, sig_f):
    print("Generating Signature")
    h = SHA256.new(data)
    rsa = RSA.importKey(key)
    signer = PKCS1_v1_5.new(rsa)
    signature = signer.sign(h)
    with open(sig_f, 'wb') as f: f.write(signature)

def verify_signature(key, data, sig_f):
    print("Verifying Signature")
    h = SHA256.new(data)
    rsa = RSA.importKey(key)
    signer = PKCS1_v1_5.new(rsa)
    with open(sig_f, 'rb') as f: signature = f.read()
    rsp = "Success" if (signer.verify(h, signature)) else " Verification Failure"
    print(rsp)

# Read all file contents
with open(key_f, 'rb') as f: key = f.read()
with open(data_f, 'rb') as f: data = f.read()

if (op == "-s"):
    # Generate Signature
    generate_signature(key, data, sig_f)
elif (op == "-v"):
    # Verify Signature
    verify_signature(key, data, sig_f)
else:
    #Error
    usage()

In [1]:
import glob
filez = raw_input("which files do you want processed?")
filez_l = filez.split()

NameError: name 'raw_input' is not defined

In [2]:
import signVerify

In [7]:
signVerify.gen_signature("privateJERS.key" ,sha1.hexdigest())

TypeError: data must be bytes-like